In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
url = 'https://www.thefork.fr/restaurant/le-pave-des-minimes-r41952/menu'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
soup.title.string

'thefork.fr'

In [3]:
print(soup.prettify())

<html>
 <head>
  <title>
   thefork.fr
  </title>
  <style>
   #cmsg{animation: A 1.5s;}@keyframes A{0%{opacity:0;}99%{opacity:0;}100%{opacity:1;}}
  </style>
 </head>
 <body style="margin:0">
  <p id="cmsg">
   Please enable JS and disable any ad blocker
  </p>
  <script data-cfasync="false">
   var dd={'rt':'c','cid':'AHrlqAAAAAMAtrNo49cxx3IAwTEw9A==','hsh':'4980A61279181687DE605B235F81B9','t':'bv','s':326,'e':'e198098f28f00f2267f3b2b4ffc3cfd7e62a2dba50f7fc921be5e47e23824d0f','host':'geo.captcha-delivery.com'}
  </script>
  <script data-cfasync="false" src="https://ct.captcha-delivery.com/c.js">
  </script>
 </body>
</html>



In [4]:
print(soup.body.text)


Please enable JS and disable any ad blocker


In [12]:
soup.find_all('title')

[<title>thefork.fr</title>]